In [1]:
from atmospheric_explorer.cams_interfaces import InversionOptimisedGreenhouseGas, EAC4Instance
from atmospheric_explorer.shapefile import ShapefilesDownloader
from atmospheric_explorer.utils import get_local_folder
from atmospheric_explorer.units_conversion import convert_units_array
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import os
from glob import glob
import xarray as xr
from datetime import datetime
import geopandas as gpd
from shapely.geometry import mapping
import plotly.graph_objects as go
import shutil
import numpy as np
import statsmodels.stats.api as sms
import pandas as pd
from plotly.subplots import make_subplots
from math import ceil
from atmospheric_explorer.plotting_apis import line_with_ci_subplots, clip_and_concat_countries, ghg_surface_satellite_yearly_plot, eac4_anomalies_plot, eac4_hovmoeller_latitude_plot, eac4_hovmoeller_levels_plot

from atmospheric_explorer.utils import hex_to_rgb
from atmospheric_explorer.data_transformations import confidence_interval

2023-08-07 14:52:02,788 INFO Loaded constants from file constants.cfg
2023-08-07 14:52:02,793 INFO Evaluating arithmetic formulas in config
2023-08-07 14:52:02,797 DEBUG Parsing 1
2023-08-07 14:52:02,800 DEBUG Parsing 1
2023-08-07 14:52:02,803 DEBUG Parsing 1
2023-08-07 14:52:02,806 DEBUG Parsing 1
2023-08-07 14:52:02,809 DEBUG Parsing 1
2023-08-07 14:52:02,814 DEBUG Parsing 1
2023-08-07 14:52:02,817 DEBUG Parsing 1
2023-08-07 14:52:02,820 DEBUG Parsing 1
2023-08-07 14:52:02,824 DEBUG Parsing 1
2023-08-07 14:52:02,827 DEBUG Parsing 1
2023-08-07 14:52:02,833 DEBUG Parsing 1
2023-08-07 14:52:02,837 DEBUG Parsing 1
2023-08-07 14:52:02,840 DEBUG Parsing 1
2023-08-07 14:52:02,843 DEBUG Parsing 1
2023-08-07 14:52:02,849 DEBUG Parsing 1
2023-08-07 14:52:02,854 DEBUG Parsing 1
2023-08-07 14:52:02,856 DEBUG Parsing 1
2023-08-07 14:52:02,859 DEBUG Parsing 1
2023-08-07 14:52:02,865 DEBUG Parsing 1
2023-08-07 14:52:02,869 DEBUG Parsing 1
2023-08-07 14:52:02,872 DEBUG Parsing 1
2023-08-07 14:52:02,

In [2]:
years=[str(y) for y in range(2020, 2021)]
months=[
    '01', '02', '03',
    '04', '05', '06',
    '07', '08', '09',
    '10', '11', '12'
]
countries = ['Italy']
data_variable = 'carbon_dioxide'
var_name = 'flux_foss'

In [3]:
manager = InversionOptimisedGreenhouseGas(
    data_variables='carbon_dioxide',
    file_format='zip',
    quantity='surface_flux',
    input_observations='surface',
    time_aggregation='monthly_mean',
    year=[ '2020'
    ],
    month=[
        '01', '02', '03',
        '04', '05', '06',
        '07', '08', '09',
        '10', '11', '12'
    ]
)
manager.download()

2023-08-07 14:52:52,054 INFO Created folder C:\Users\User\AppData\Local\AtmosphericExplorer\data\global_greenhouse_gas_inversion
2023-08-07 14:52:52,061 INFO Created folder C:\Users\User\AppData\Local\AtmosphericExplorer\data\global_greenhouse_gas_inversion\data_1
2023-08-07 14:52:54,296 INFO Welcome to the CDS
2023-08-07 14:52:54,299 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-global-greenhouse-gas-inversion
2023-08-07 14:52:54,416 INFO Request is queued
2023-08-07 14:52:55,535 INFO Request is completed
2023-08-07 14:52:55,542 INFO Downloading https://download-0004-ads-clone.copernicus-climate.eu/cache-compute-0004/cache/data7/dataset-cams-global-greenhouse-gas-inversion-a1a90e06-e19b-4ebe-b01c-f1504714260b.zip to C:\Users\User\AppData\Local\AtmosphericExplorer\data\global_greenhouse_gas_inversion\data_1\data_1.zip (2.5M)
2023-08-07 14:52:59,351 INFO Download rate 671.3K/s 
2023-08-07 14:52:59,461 INFO Finished downloading file C:\Users\User\AppD

In [41]:
files = sorted(glob(manager.file_full_path))
df = xr.open_dataset(files[0])
area=sum(np.array(df.area[0]))
area

34864536576.0

In [15]:
manager2 = EAC4Instance(
    'total_column_ozone',
    'netcdf',
    dates_range = '2018-01-01/2018-06-01',
    time_values = '00:00'
)
manager2.download()

2023-08-07 15:03:35,335 INFO Created folder C:\Users\User\AppData\Local\AtmosphericExplorer\data\eac4
2023-08-07 15:03:35,344 INFO Created folder C:\Users\User\AppData\Local\AtmosphericExplorer\data\eac4\data_8
2023-08-07 15:03:35,484 INFO Welcome to the CDS
2023-08-07 15:03:35,486 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-global-reanalysis-eac4
2023-08-07 15:03:35,671 INFO Request is queued
2023-08-07 15:03:36,872 INFO Request is running
2023-08-07 15:03:44,330 INFO Request is completed
2023-08-07 15:03:44,334 INFO Downloading https://download-0002-ads-clone.copernicus-climate.eu/cache-compute-0002/cache/data3/adaptor.mars.internal-1691413424.4268925-20167-11-84584a66-db3a-46dd-a348-a1e800aeb898.nc to C:\Users\User\AppData\Local\AtmosphericExplorer\data\eac4\data_8\data_8.nc (33.5M)
2023-08-07 15:03:54,835 INFO Download rate 3.2M/s   
2023-08-07 15:03:54,925 INFO Finished downloading file C:\Users\User\AppData\Local\AtmosphericExplorer\data\eac

In [16]:
df2 = xr.open_dataset(manager2.file_full_path)
df2

<xarray.Dataset>
Dimensions:    (longitude: 480, latitude: 241, time: 152)
Coordinates:
  * longitude  (longitude) float32 0.0 0.75 1.5 2.25 ... 357.0 357.8 358.5 359.2
  * latitude   (latitude) float32 90.0 89.25 88.5 87.75 ... -88.5 -89.25 -90.0
  * time       (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-06-01
Data variables:
    gtco3      (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-08-07 13:03:45 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...